# 더해보기 - Batch Normalization

- https://shuuki4.wordpress.com/2016/01/13/batch-normalization-%EC%84%A4%EB%AA%85-%EB%B0%8F-%EA%B5%AC%ED%98%84/

이 기법은 overfitting 뿐만 아니라 학습 속도 또한 향상시켜 주는 장점이 있다.

In [1]:
import tensorflow as tf
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Instructions for updating:
Use the retry module or similar alternatives.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

is_training = tf.placeholder(tf.bool)

W1 = tf.Variable(tf.random_normal([784, 256], stddev=0.01))
b1 = tf.Variable(tf.zeros([256]))
L1 = tf.nn.relu(tf.add(tf.matmul(X, W1), b1))
L1 = tf.layers.batch_normalization(L1, training=is_training)

W2 = tf.Variable(tf.random_normal([256, 256], stddev=0.01))
b2 = tf.Variable(tf.zeros([256]))
L2 = tf.nn.relu(tf.add(tf.matmul(L1, W2), b2))
L2 = tf.layers.batch_normalization(L2, training=is_training)

W3 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
b3 = tf.Variable(tf.zeros([10]))
model = tf.add(tf.matmul(L2, W3), b3)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
opt = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

In [7]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

print('batch size : %s' % batch_size)
print('total batch size : %s' % total_batch)

for epoch in range(10):
    total_cost = 0

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        _, cost_val = sess.run([opt, cost], feed_dict={X: batch_xs, Y: batch_ys, is_training: True})
        total_cost += cost_val
        
    print('Epoch: %04d' % (epoch + 1), 'Avg. cost = {:.3f}'.format(total_cost/total_batch))

print('Complete!')

batch size : 100
total batch size : 550
Epoch: 0001 Avg. cost = 0.212
Epoch: 0002 Avg. cost = 0.113
Epoch: 0003 Avg. cost = 0.082
Epoch: 0004 Avg. cost = 0.066
Epoch: 0005 Avg. cost = 0.058
Epoch: 0006 Avg. cost = 0.053
Epoch: 0007 Avg. cost = 0.046
Epoch: 0008 Avg. cost = 0.040
Epoch: 0009 Avg. cost = 0.040
Epoch: 0010 Avg. cost = 0.036
Complete!


In [9]:
is_correct = tf.equal(tf.argmax(model, axis=1), tf.argmax(Y, axis=1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('Accuracy : %s' % sess.run(accuracy, 
                                 feed_dict={X: mnist.test.images, Y: mnist.test.labels, is_training: False}))

Accuracy : 0.7466


# 특이사항

- batch_normalization만 했을 때정확도가 70~80% 정도밖에 안됨...